<a href="https://colab.research.google.com/github/m-rafiul-islam/student-learning-analytics-lab/blob/main/student_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://archive.ics.uci.edu/static/public/349/open+university+learning+analytics+dataset.zip" -O data-oulad.zip
!unzip  data-oulad.zip -d oulad-data

--2026-02-14 21:17:16--  https://archive.ics.uci.edu/static/public/349/open+university+learning+analytics+dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data-oulad.zip’

data-oulad.zip          [    <=>             ]  44.58M  70.7MB/s    in 0.6s    

2026-02-14 21:17:17 (70.7 MB/s) - ‘data-oulad.zip’ saved [46748244]



Archive:  data-oulad.zip
  inflating: oulad-data/assessments.csv  
  inflating: oulad-data/courses.csv  
  inflating: oulad-data/studentAssessment.csv  
  inflating: oulad-data/studentInfo.csv  
  inflating: oulad-data/studentRegistration.csv  
  inflating: oulad-data/studentVle.csv  
  inflating: oulad-data/vle.csv      
  inflating: oulad-data/OULAD.names  


#

In [ ]:
# ===========================================
# OULAD Colab Template: Student Risk Prediction
# ===========================================
# Goal: Predict at-risk students (Fail/Withdrawn) using early activity + assessments
# Data: OULAD CSVs (studentInfo, studentRegistration, assessments, studentAssessment,
#       studentVle, vle, courses)
# -------------------------------------------

# ============== 0) SETUP ==============
!pip -q install xgboost shap

import os
import zipfile
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score,
    confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

RANDOM_STATE = 42

# ============== 1) UPLOAD & UNZIP DATA ==============
# Upload `oulad.zip` to Colab (Files panel), then run this cell.

DATA_DIR = "/content/oulad_data"
os.makedirs(DATA_DIR, exist_ok=True)

zip_path = "/content/oulad.zip"
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(DATA_DIR)
    print("✅ Unzipped oulad.zip into:", DATA_DIR)
else:
    raise FileNotFoundError("❌ /content/oulad.zip not found. Upload oulad.zip to Colab first.")

# Helper: locate CSVs even if nested in a folder
def find_csv(filename, base_dir):
    for root, _, files in os.walk(base_dir):
        if filename in files:
            return os.path.join(root, filename)
    return None

needed = [
    "studentInfo.csv","studentRegistration.csv","assessments.csv",
    "studentAssessment.csv","studentVle.csv","vle.csv","courses.csv"
]
paths = {f: find_csv(f, DATA_DIR) for f in needed}
missing = [k for k,v in paths.items() if v is None]
if missing:
    raise FileNotFoundError(f"❌ Missing files: {missing}\nCheck the zip contents / folder structure.")
print("✅ All required CSVs found.")

# ============== 2) LOAD TABLES ==============
studentInfo         = pd.read_csv(paths["studentInfo.csv"])
studentRegistration = pd.read_csv(paths["studentRegistration.csv"])
assessments         = pd.read_csv(paths["assessments.csv"])
studentAssessment   = pd.read_csv(paths["studentAssessment.csv"])
studentVle          = pd.read_csv(paths["studentVle.csv"])
vle                 = pd.read_csv(paths["vle.csv"])
courses             = pd.read_csv(paths["courses.csv"])

print("studentInfo:", studentInfo.shape)
print("studentRegistration:", studentRegistration.shape)
print("assessments:", assessments.shape)
print("studentAssessment:", studentAssessment.shape)
print("studentVle:", studentVle.shape)
print("vle:", vle.shape)
print("courses:", courses.shape)

studentInfo.head()

# ============== 3) DEFINE TARGET LABEL ==============
# final_result: Pass, Fail, Withdrawn, Distinction
# Binary: at risk = Fail or Withdrawn (customize as needed)

TARGET_COL = "final_result"
AT_RISK = {"Fail", "Withdrawn"}

df_target = studentInfo[["id_student","code_module","code_presentation", TARGET_COL]].copy()
df_target["y_at_risk"] = df_target[TARGET_COL].isin(AT_RISK).astype(int)

print(df_target["y_at_risk"].value_counts())

# ============== 4) EARLY-WEEK FEATURE ENGINEERING ==============
# Use only activity/assessments up to EARLY_WEEK (weeks from start).
EARLY_WEEK = 4
CUTOFF_DAYS = EARLY_WEEK * 7

# ---- 4A) Registration features ----
reg_feats = studentRegistration[[
    "id_student","code_module","code_presentation",
    "date_registration","date_unregistration"
]].copy()

reg_feats["registered_early"] = (reg_feats["date_registration"] <= 0).astype(int)
reg_feats["unregistered"] = reg_feats["date_unregistration"].notna().astype(int)
reg_feats["unregistered_by_cutoff"] = (
    reg_feats["date_unregistration"].notna() & (reg_feats["date_unregistration"] <= CUTOFF_DAYS)
).astype(int)

# ---- 4B) Assessment aggregates up to cutoff ----
assess_merged = studentAssessment.merge(
    assessments[["id_assessment","code_module","code_presentation","date","weight","assessment_type"]],
    on="id_assessment",
    how="left"
)

assess_early = assess_merged[assess_merged["date"] <= CUTOFF_DAYS].copy()

assess_feats = assess_early.groupby(["id_student","code_module","code_presentation"]).agg(
    n_assess_submitted=("score","count"),
    mean_score=("score","mean"),
    max_score=("score","max"),
    min_score=("score","min"),
    sum_weight=("weight","sum")
).reset_index()

# ---- 4C) VLE click aggregates up to cutoff ----
vle_merged = studentVle.merge(
    vle[["id_site","activity_type"]],
    on="id_site",
    how="left"
)

vle_early = vle_merged[vle_merged["date"] <= CUTOFF_DAYS].copy()

vle_total = vle_early.groupby(["id_student","code_module","code_presentation"]).agg(
    total_clicks=("sum_click","sum"),
    active_days=("date","nunique")
).reset_index()

vle_by_type = (vle_early
               .groupby(["id_student","code_module","code_presentation","activity_type"])["sum_click"]
               .sum()
               .reset_index())

vle_pivot = vle_by_type.pivot_table(
    index=["id_student","code_module","code_presentation"],
    columns="activity_type",
    values="sum_click",
    fill_value=0
).reset_index()

# ============== 5) BUILD MASTER MODEL TABLE ==============
demo_cols = [
    "id_student","code_module","code_presentation",
    "gender","region","highest_education","imd_band",
    "age_band","num_of_prev_attempts","studied_credits",
    "disability"
]

df = studentInfo[demo_cols].merge(
    df_target[["id_student","code_module","code_presentation","y_at_risk"]],
    on=["id_student","code_module","code_presentation"],
    how="inner"
)

df = df.merge(reg_feats, on=["id_student","code_module","code_presentation"], how="left")
df = df.merge(assess_feats, on=["id_student","code_module","code_presentation"], how="left")
df = df.merge(vle_total, on=["id_student","code_module","code_presentation"], how="left")
df = df.merge(vle_pivot, on=["id_student","code_module","code_presentation"], how="left")

# Fill missing numeric engineered features with 0
num_all = df.select_dtypes(include=[np.number]).columns.tolist()
num_all.remove("y_at_risk")
df[num_all] = df[num_all].fillna(0)

print("Master df:", df.shape)
df.head()

# ============== 6) TRAIN/TEST SPLIT ==============
X = df.drop(columns=["y_at_risk"])
y = df["y_at_risk"]

# remove identifier(s)
X = X.drop(columns=["id_student"], errors="ignore")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print("Categorical cols:", cat_cols)
print("Numeric cols:", len(num_cols))

# ============== 7) PREPROCESSING PIPELINE ==============
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# ============== 8) LOGISTIC REGRESSION BASELINE ==============
logreg = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=2000))
])

logreg.fit(X_train, y_train)
p = logreg.predict_proba(X_test)[:, 1]
yhat = (p >= 0.5).astype(int)

print("\n=== Logistic Regression ===")
print("AUC:", roc_auc_score(y_test, p))
print("F1:", f1_score(y_test, yhat))
print("Accuracy:", accuracy_score(y_test, yhat))
print("Confusion Matrix:\n", confusion_matrix(y_test, yhat))
print("\nReport:\n", classification_report(y_test, yhat, digits=3))

# ============== 9) RANDOM FOREST ==============
rf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=400, random_state=RANDOM_STATE,
        class_weight="balanced_subsample", n_jobs=-1
    ))
])

rf.fit(X_train, y_train)
p = rf.predict_proba(X_test)[:, 1]
yhat = (p >= 0.5).astype(int)

print("\n=== Random Forest ===")
print("AUC:", roc_auc_score(y_test, p))
print("F1:", f1_score(y_test, yhat))
print("Accuracy:", accuracy_score(y_test, yhat))
print("Confusion Matrix:\n", confusion_matrix(y_test, yhat))

# ============== 10) OPTIONAL: XGBOOST ==============
from xgboost import XGBClassifier

xgb = Pipeline(steps=[
    ("prep", preprocess),
    ("model", XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=RANDOM_STATE,
        eval_metric="logloss",
        n_jobs=-1
    ))
])

xgb.fit(X_train, y_train)
p = xgb.predict_proba(X_test)[:, 1]
yhat = (p >= 0.5).astype(int)

print("\n=== XGBoost ===")
print("AUC:", roc_auc_score(y_test, p))
print("F1:", f1_score(y_test, yhat))
print("Accuracy:", accuracy_score(y_test, yhat))
print("Confusion Matrix:\n", confusion_matrix(y_test, yhat))

# ============== 11) OPTIONAL: SHAP (XGBoost) ==============
import shap

prep = xgb.named_steps["prep"]
model = xgb.named_steps["model"]

X_test_t = prep.transform(X_test)

ohe = prep.named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(cat_cols).tolist()
feature_names = cat_feature_names + num_cols

sample_n = min(2000, X_test_t.shape[0])
idx = np.random.RandomState(RANDOM_STATE).choice(X_test_t.shape[0], sample_n, replace=False)
X_shap = X_test_t[idx]

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_shap)

shap.summary_plot(shap_values, features=X_shap, feature_names=feature_names, show=False)
plt.tight_layout()
plt.show()

# ============== 12) SAVE MASTER TABLE USED FOR MODELING ==============
out_path = f"/content/oulad_master_week{EARLY_WEEK}.csv"
df.to_csv(out_path, index=False)
print("✅ Saved:", out_path)

# Tip: change EARLY_WEEK to 2,4,6,8 and compare AUC/F1 for "early warning" analysis.
